In [1]:
import pandas as pd

import base64
import numpy as np
from io import BytesIO
from PIL import Image

import logging
import sys

import json
import requests
import s3fs
import posixpath
import os

import joblib

D:\Utils\Anaconda3\envs\p36\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
data_start = "2019-06-09 07:12"
data_end = "2019-09-17"
location = "stockholm"
dest_folder = "<your destination s3 path>"
authorization = "<your EWD authorization token>"

In [ ]:
data_range = pd.date_range(data_start, data_end, freq="1min")

In [ ]:
def download_reftime(timestamp, location, dest_folder, authorization, overwrite=False):
    try:
        s3 = s3fs.S3FileSystem()
        dest_file = posixpath.join(dest_folder, timestamp.strftime("%Y%m"), "%s.npy" % timestamp.strftime("%Y%m%d%H%M"))
        
        if s3.exists(dest_file) and not overwrite:
            return (timestamp, None)
        
        method = "GET"
        url = "https://demo-apim.westeurope.cloudapp.azure.com/api_secure/PrecipitationAPI/3.0.0/weather/precipitation/at/%s?location=%s" % (timestamp.strftime("%Y%m%d%H%M"), location)

        headers = {"accept": "application/json",
                   "Authorization": authorization}
        response = requests.request(method=method, url=url, headers=headers)
        if response.status_code != 200:
            return (timestamp, response.status_code)
        
        resp = json.loads(response.content.decode("utf-8"))
        
        b64png = resp['canvas'][len("data:image/png;base64,"):]
        dec = base64.b64decode(b64png)
        image = Image.open(BytesIO(dec))
        image = image.resize((64, 61))
        img = np.array(image)
        img = np.flipud(img)        
        
        with s3.open(dest_file, mode='wb') as f:
            np.save(f, img)

        return (timestamp, 200)
    except Exception as e:
        return (timestamp, str(e))


In [ ]:
with joblib.parallel_backend("threading"):
    results = joblib.Parallel(n_jobs=4)(
                    joblib.delayed(download_reftime)(timestamp, location, dest_folder, authorization)
                        for timestamp in data_range)